In [11]:

from pathlib import Path
root_path = "/home/minghuan/ppt_learning/logs/images/2025-05-06_16-43-38"
rgb_only_table_path = "/home/minghuan/ppt_learning/logs/images/2025-05-06_16-43-38/color_1/1746521024751.png"
depth_only_table_path = "/home/minghuan/ppt_learning/logs/images/2025-05-06_16-43-38/depth_1/1746521024751.npy"
output_path = Path("/home/minghuan/ppt_learning/logs/images/2025-05-06_16-43-38/output_pcds")
table_to_robot_base_npy = "/home/minghuan/ppt_learning/logs/table_to_robot.npy"
image_with_aruco_marker = "/home/minghuan/ppt_learning/logs/images/2025-05-06_16-43-38/color_1/1746521024751.png"
image_with_aruco_marker = "/home/minghuan/ppt_learning/tests/cab.png"
gt_obj_file_path = "/home/minghuan/ppt_learning/logs/gt/T_block_left/T_block_left.obj"

In [12]:
import cv2
import numpy as np
import open3d as o3d
from pathlib import Path
import os
os.environ["DISPLAY"] = ":0"

def depth_to_pointcloud(rgb_image, depth_map, intrinsics, depth_max_threshold=1.3):
    """
    将RGB图像和深度图转换为彩色点云
    
    参数:
        rgb_image: RGB图像，numpy数组 (H, W, 3)
        depth_map: 深度图，numpy数组 (H, W)
        intrinsics: 相机内参矩阵 (3, 3)
    
    返回:
        point_cloud: Open3D点云对象
    """
    # 获取图像尺寸
    height, width = depth_map.shape
    
    # 创建像素坐标网格
    v, u = np.mgrid[0:height, 0:width]
    
    # 重塑为列向量
    u = u.reshape(-1)
    v = v.reshape(-1)
    depth = depth_map.reshape(-1)
    
    # 过滤无效的深度值（0或负值）
    valid_indices = np.logical_and(depth > 0,  depth < depth_max_threshold)
    u = u[valid_indices]
    v = v[valid_indices]
    depth = depth[valid_indices]
    
    # 获取相机内参
    fx = intrinsics[0, 0]  # 焦距x
    fy = intrinsics[1, 1]  # 焦距y
    cx = intrinsics[0, 2]  # 光心x
    cy = intrinsics[1, 2]  # 光心y
    
    # 计算3D坐标
    x = (u - cx) * depth / fx
    y = (v - cy) * depth / fy
    z = depth
    
    # 创建点云
    points = np.vstack((x, y, z)).T
    
    # 从RGB图像获取颜色
    colors = rgb_image.reshape(-1, 3)[valid_indices] / 255.0
    
    # 创建Open3D点云对象
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    return point_cloud, points

def get_intr_matrix(only_table_intrinsics):
    """
    获取相机内参矩阵
    """
    fx = only_table_intrinsics["arr_0"][0]["fx"]
    fy = only_table_intrinsics["arr_0"][0]["fy"]
    cx = only_table_intrinsics["arr_0"][0]["ppx"]
    cy = only_table_intrinsics["arr_0"][0]["ppy"]
    intrinsics = np.array([[fx, 0, cx],
                           [0, fy, cy],
                           [0, 0, 1]])
    return intrinsics   

def estimate_pose(image, charuco_dict, intrinsics_matrix, dist_coeffs, board):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = cv2.aruco.detectMarkers(gray, charuco_dict)
    
    if len(corners) > 0:
        ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board) # can not pass
        if charuco_ids is not None and len(charuco_corners) > 3:
            valid, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, intrinsics_matrix, dist_coeffs, None, None)
            if valid:
                R_target2cam = cv2.Rodrigues(rvec)[0]
                t_target2cam = tvec.reshape(3, 1)
                target2cam = np.eye(4)
                target2cam[:3, :3] = R_target2cam
                target2cam[:3, 3] = t_target2cam.reshape(-1)
                return np.linalg.inv(target2cam)
    return None

def load_obj_as_pointcloud(obj_file_path, sample_points=0):
    """
    从OBJ文件中加载3D模型并转换为点云
    
    参数:
        obj_file_path: OBJ文件路径
        sample_points: 采样点数量，若为0则使用所有顶点，若大于0则对网格进行采样
        
    返回:
        point_cloud: Open3D点云对象
    """
    print(f"正在加载OBJ文件: {obj_file_path}")
    
    # 加载OBJ文件为网格
    mesh = o3d.io.read_triangle_mesh(obj_file_path)
    
    # 确保网格有法线，如果没有则计算法线
    if not mesh.has_vertex_normals():
        mesh.compute_vertex_normals()
    
    print(f"网格加载完成: {len(mesh.vertices)}个顶点, {len(mesh.triangles)}个三角形")
    
    # 从网格中获取点云
    if sample_points > 0:
        print(f"对网格进行采样: {sample_points}个点")
        point_cloud = mesh.sample_points_uniformly(number_of_points=sample_points)
    else:
        # 使用网格顶点创建点云
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = mesh.vertices
        point_cloud.colors = mesh.vertex_colors if mesh.has_vertex_colors() else None
        point_cloud.normals = mesh.vertex_normals if mesh.has_vertex_normals() else None
    
    print(f"点云创建完成: {len(point_cloud.points)}个点")
    
    return point_cloud



## Load data

In [13]:
import os
os.environ["DISPLAY"] = ":0"
only_table_index = 1
depth_threshold = 2
root_path = Path(root_path)
# only_table_intrinsics = np.load(root_path / "only_table/intrinsics.npz", allow_pickle=True)
only_table_intrinsics = np.load(root_path / "intrinsics.npz", allow_pickle=True)
# rgb_only_table_path = list((root_path / "only_table/color_0").glob("*.png"))[only_table_index]
# depth_only_table_path = list((root_path / "only_table/depth_0").glob("*.npy"))[only_table_index]
# print(cv2.imread(str(rgb_only_table_path)))
rgb_only_table = cv2.cvtColor(cv2.imread(str(rgb_only_table_path)), cv2.COLOR_BGR2RGB)
depth_only_table = np.load(str(depth_only_table_path))
intrinsics = get_intr_matrix(only_table_intrinsics)
pcd_table_only, pcd_np_clean_table = depth_to_pointcloud(rgb_only_table, depth_only_table, intrinsics, depth_threshold)
to_vis = []
to_vis.append(pcd_table_only)
frame_base = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
to_vis.append(frame_base)
o3d.visualization.draw_geometries(to_vis)


[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


## Get GT

In [8]:
from scipy.spatial.transform import Rotation as R

gt_pcd = load_obj_as_pointcloud(gt_obj_file_path, sample_points=50000)
translation = np.array([(0.3395, 0, 0.529)])
# 创建平移矩阵
transform = np.eye(4)  # 创建4x4单位矩阵
transform[:3, 3] = translation  # 设置平移部分
# 应用变换
gt_pcd.transform(transform)
transform = np.eye(4)  # 创建4x4单位矩阵
transform[:3, :3] = R.from_euler('xyz', [-90, 0, 0], degrees=True).as_matrix()
gt_pcd.transform(transform)

vis = []
frame_base = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
vis.append(gt_pcd)
vis.append(frame_base)
o3d.visualization.draw_geometries(vis)


正在加载OBJ文件: /home/minghuan/ppt_learning/logs/gt/T_block_left/T_block_left.obj
网格加载完成: 66个顶点, 40个三角形
对网格进行采样: 50000个点
点云创建完成: 50000个点
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


## Init Aruco

In [9]:
import cv2
import numpy as np

def estimate_pose(image, charuco_dict, intrinsics_matrix, dist_coeffs, board):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detector_params = cv2.aruco.DetectorParameters()
    aruco_detector = cv2.aruco.ArucoDetector(charuco_dict, detector_params)
    corners, ids, _ = aruco_detector.detectMarkers(image)

    charucodetector = cv2.aruco.CharucoDetector(board)
    charuco_corners, charuco_ids, marker_corners, marker_ids = charucodetector.detectBoard(image)
    # print('Detected markers: ', ids)
    print(charuco_corners)
    print(len(marker_corners))
    if charuco_ids is not None and len(charuco_corners) > 3:
        valid, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, intrinsics_matrix, dist_coeffs, None, None)
        if valid:
            R_target2cam = cv2.Rodrigues(rvec)[0]
            t_target2cam = tvec.reshape(3, 1)
            target2cam = np.eye(4)
            target2cam[:3, :3] = R_target2cam
            target2cam[:3, 3] = t_target2cam.reshape(-1)
            return np.linalg.inv(target2cam)
    return None

image = cv2.imread("/home/minghuan/ppt_learning/logs/photos/2025-05-06_18-20-48/color_1/1746526856232.png")
charuco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard((12, 10), 0.04, 0.03, charuco_dict)
cam_2_marker_init = estimate_pose(image, charuco_dict, intrinsics, np.zeros(5), board)
cam_2_marker_init

[[[789.46906 427.7479 ]]

 [[699.93463 406.55243]]

 [[666.4892  338.48077]]

 [[649.64514 337.85358]]

 [[631.5763  336.5893 ]]

 [[787.6045  311.09576]]

 [[787.53455 294.72025]]]
19


array([[-9.99163979e-01, -4.08807455e-02, -3.29391964e-04,
         3.88910343e-01],
       [ 3.87968063e-02, -9.50709366e-01,  3.07646727e-01,
        -4.65329402e-01],
       [-1.28899836e-02,  3.07376749e-01,  9.51500595e-01,
        -1.97107100e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Align by ICP

In [10]:
from matplotlib import pyplot as plt
import open3d as o3d
import copy

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

draw_registration_result(pcd_table_only, gt_pcd, cam_2_marker_init)
print("Initial alignment")
print(cam_2_marker_init)
threshold = 0.01
cam_2_marker_evaluation = o3d.pipelines.registration.evaluate_registration(
    pcd_table_only, gt_pcd, threshold, cam_2_marker_init)
print(cam_2_marker_evaluation)
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd_table_only, gt_pcd, threshold, cam_2_marker_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=20000))

print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(pcd_table_only, gt_pcd, reg_p2p.transformation)
camera_2_marker = reg_p2p.transformation
np.save(Path(rgb_only_table_path).parent.parent / "camera_2_table.npy", camera_2_marker)

[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
Initial alignment
[[-9.99163979e-01 -4.08807455e-02 -3.29391964e-04  3.88910343e-01]
 [ 3.87968063e-02 -9.50709366e-01  3.07646727e-01 -4.65329402e-01]
 [-1.28899836e-02  3.07376749e-01  9.51500595e-01 -1.97107100e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
Apply point-to-point ICP
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
Transformation is:
[[-9.99163979e-01 -4.08807455e-02 -3.29391964e-04  3.88910343e-01]
 [ 3.87968063e-02 -9.50709366e-01  3.07646727e-01 -4.65329402e-01]
 [-1.28899836e-02  3.07376749e-0